<a href="https://colab.research.google.com/github/raypratik9/Fraud-detection/blob/main/Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#ML libraries
import tensorflow as tf
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

#Metrics Libraries
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix



#Misc libraries
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Fraud.csv')
df.shape

(6362620, 11)

In [ ]:
df.tail()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.0,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.0,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.0,C2080388513,0.00,0.00,1,0
6362619,743,CASH_OUT,850002.52,C1280323807,850002.52,0.0,C873221189,6510099.11,7360101.63,1,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [ ]:
print("Total Fraud-Transaction:", len(df[df['isFraud']==1]))
print("Total Valid-Transaction:", len(df[df['isFraud']==0]))

Total Fraud-Transaction: 8213
Total Valid-Transaction: 6354407


#Data Cleaning

NULL or NaN values in the dataset




In [ ]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

Treating Outliers

In [ ]:
from scipy import stats

In [ ]:
z = np.abs(stats.zscore(df[['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']]))
print(z)

[[2.81559923e-01 2.29810037e-01 2.37621696e-01 3.23813895e-01
  3.33411405e-01]
 [2.94767262e-01 2.81359380e-01 2.85812295e-01 3.23813895e-01
  3.33411405e-01]
 [2.97554804e-01 2.88653782e-01 2.92441707e-01 3.23813895e-01
  3.33411405e-01]
 ...
 [1.01539526e+01 1.89649113e+00 2.92441707e-01 3.03665258e-01
  1.40302700e+00]
 [1.10976490e+00 5.58104668e-03 2.92441707e-01 3.23813895e-01
  3.33411405e-01]
 [1.10976490e+00 5.58104668e-03 2.92441707e-01 1.59138312e+00
  1.66981230e+00]]


In [ ]:
threshold = 3
print(np.where(z > 3))

(array([     84,      88,      89, ..., 6362599, 6362616, 6362617]), array([4, 4, 4, ..., 0, 0, 0]))


In [ ]:
print(z[84][4])

4.883935818423778


In [ ]:
df_new = df[(z < 3).all(axis=1)]

In [ ]:
df.shape

(6362620, 11)

In [ ]:
df_new.shape

(6090589, 11)

In [ ]:
dummy_type=pd.get_dummies(df_new['type'])
dummy_type.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6090589 entries, 0 to 6362619
Data columns (total 5 columns):
 #   Column    Dtype
---  ------    -----
 0   CASH_IN   uint8
 1   CASH_OUT  uint8
 2   DEBIT     uint8
 3   PAYMENT   uint8
 4   TRANSFER  uint8
dtypes: uint8(5)
memory usage: 75.5 MB


In [ ]:
df_new=pd.concat([df_new,dummy_type],axis=1)

In [ ]:
print("Total Fraud-Transaction:", len(df_new[df_new['isFraud']==1]))
print("Total Valid-Transaction:", len(df_new[df_new['isFraud']==0]))

Total Fraud-Transaction: 6485
Total Valid-Transaction: 6084104


In [ ]:
df_new.groupby('isFraud').mean()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
isFraud,,,,,,,,,,,,
0,242.839546,144631.029706,454638.850957,474521.117296,818627.120285,907732.311040,0.000000,0.200797,0.361156,0.006695,0.35362,0.077731
1,366.257209,465158.785314,463726.430891,458.779644,399102.775224,634305.955593,0.000463,0.000000,0.499306,0.000000,0.00000,0.500694


In [ ]:
fraud=df_new[df_new['isFraud']==1]
legit=df_new[df_new['isFraud']==0]

In [ ]:
import plotly.graph_objects as go

#Checking for balance in target
fig = go.Figure(data=[go.Pie(labels=['Not Fraud','Fraud'], values=df_new['isFraud'].value_counts())])
fig.show()

In [ ]:
#Getting the max size
max_size = df_new['isFraud'].value_counts().max()

#Balancing the target label
lst = [df_new]
for class_index, group in df_new.groupby('isFraud'):
    lst.append(group.sample(max_size-len(group), replace=True))
df_new = pd.concat(lst)

In [ ]:
#Checking the balanced target
fig = go.Figure(data=[go.Pie(labels=['Not Fraud','Fraud'], values=df_new['isFraud'].value_counts())])
fig.show()

In [ ]:
df_new.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,0,0,0,1,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,0,0,0,1,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,0,0,0,0,1
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,0,1,0,0,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,0,0,0,1,0


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Splitting dependent and independent variable
df_new_1=df_new.copy()
X=df_new_1.drop('isFraud',axis=1)
y=df_new_1['isFraud']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=111)

#Standardizing the numerical columns
col_names=['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']
features_train = X_train[col_names]
features_test = X_test[col_names]
scaler = StandardScaler().fit(features_train.values)
features_train = scaler.transform(features_train.values)
features_test = scaler.transform(features_test.values)
X_train[col_names] = features_train
X_test[col_names] =features_test

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [ ]:
#Dropping unnecessary columns
X_train=X_train.drop(['nameOrig','nameDest','isFlaggedFraud'],axis=1)
X_train = X_train.reset_index(drop=True)

X_test=X_test.drop(['nameOrig','nameDest','isFlaggedFraud'],axis=1)
X_test = X_test.reset_index(drop=True)

#Model Building

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=20,criterion='entropy', random_state=0,max_depth=10)
print(clf.fit(x_train,y_train))

NameError: ignored

In [ ]:
y_pred = clf.predict(x_test)
from sklearn.metrics import classification_report
print('Classifcation report:\n', classification_report(y_test, y_pred))